In the code, we tried to formulate and estimate 3 types of discrete choice models: MNL, mixed logit (with 3 parameter specifications), and nested logit. Note that:
1. The parameter for $t_\textit{shift}$ is non-significant no matter how we specify the model, which is detailed in the "Model formulation & parameter estimation" section.
2. Pylogit sometimes results in negative $\sigma$ parameters when estimating a mixed logit model, but we can take the absolute values of estimates (as mentioned by Dr. Brathwaite, "...PyLogit performed an unconstrained maximization, and the normal distribution is symmetric, so it doesn't matter if we show multiply by negative one or not.").
3. Income level is the only one individual characteristic that is incorporated into our model, by estimating two parameters for respondents with low income ($<\$50\rm k$) and with high income ($\geq\$50\rm k$), respectively.

In [1]:
import pandas as pd
import numpy as np
import pylogit as pl
from collections import OrderedDict
import warnings
warnings.filterwarnings('ignore')

# Data processing

In [2]:
df = pd.read_csv('CE264_April 21, 2022_17.52.csv', skiprows=[1, 2]).fillna('')
df

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,F-2-1-4.2,F-3-2-4.0,F-3-2-4.1,F-3-2-4.2,F-4-1-4.0,F-4-1-4.1,F-4-1-4.2,F-5-1-4.0,F-5-1-4.1,F-5-1-4.2
0,2022-04-05 18:49:43,2022-04-05 18:53:39,Survey Preview,,100,236,True,2022-04-05 18:53:40,R_24Csf53B38FuyLP,,...,,,,,,,,40.0,60.0,90.0
1,2022-04-05 19:18:10,2022-04-05 19:18:21,Survey Preview,,100,10,True,2022-04-05 19:18:22,R_2Bkxjyfqv8O9OYS,,...,,,,,,,,,,
2,2022-04-05 19:59:31,2022-04-05 19:59:32,Survey Test,,100,1,True,2022-04-05 19:59:32,R_9MtQkovIbk1yUkK,,...,,,,,,,,,,
3,2022-04-05 19:59:32,2022-04-05 19:59:32,Survey Test,,100,0,True,2022-04-05 19:59:32,R_ewX7ZEAcLT0YiAC,,...,,,,,,,,,,
4,2022-04-05 19:59:33,2022-04-05 19:59:33,Survey Test,,100,0,True,2022-04-05 19:59:33,R_0d0xD9mN9LXwSrQ,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,2022-04-14 11:05:59,2022-04-14 11:06:58,IP Address,108.30.233.119,71,58,False,2022-04-21 11:07:37,R_10IsOTECUkWVDGY,,...,,,,,,,,,,
102,2022-04-21 12:20:16,2022-04-21 12:22:55,IP Address,184.23.21.197,100,158,True,2022-04-21 12:22:55,R_1M4GCxvdn1XLNQJ,,...,,,,,,,,,,
103,2022-04-21 12:23:13,2022-04-21 12:24:54,IP Address,184.23.21.197,100,101,True,2022-04-21 12:24:55,R_1jv5sVXzZtN55rO,,...,,,,,,,,,,
104,2022-04-21 12:25:00,2022-04-21 12:26:49,IP Address,184.23.21.197,100,109,True,2022-04-21 12:26:49,R_2Crlz1V5Qyzsxwt,,...,,,,,,,,,,


In [3]:
df = df[(df['Q13']!='') & (df['F-1-1-1']!='') & (df['Q8']!='')]

In [4]:
sample = df[['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q13', 'Q14', 'Q15', 'Q16', 
             'F-1-1-1', 'F-1-1-2', 'F-1-1-3', 'F-1-1-4', 'F-1-2-1', 'F-1-2-2', 'F-1-2-3', 'F-1-2-4',
             'F-2-1-1', 'F-2-1-2', 'F-2-1-3', 'F-2-1-4', 'F-2-2-1', 'F-2-2-2', 'F-2-2-3', 'F-2-2-4',
             'F-3-1-1', 'F-3-1-2', 'F-3-1-3', 'F-3-1-4', 'F-3-2-1', 'F-3-2-2', 'F-3-2-3', 'F-3-2-4',
             'F-4-1-1', 'F-4-1-2', 'F-4-1-3', 'F-4-1-4', 'F-4-2-1', 'F-4-2-2', 'F-4-2-3', 'F-4-2-4']]

col_name = {'Q1': 'is_driver', 'Q2': 'age', 'Q3': 'gender', 'Q4': 'occ', 'Q5': 'income', 'Q6': 'hh_size',
            'Q7': 'have_driven', 'Q8': 'trip_purp', 'Q9': 'start_time', 'Q10': 'trip_freq',
            'Q13': 'choice_1', 'Q14': 'choice_2', 'Q15': 'choice_3', 'Q16': 'choice_4',
            'F-1-1-1': 't_detour_1a', 'F-1-1-2': 't_cong_1a', 'F-1-1-3': 't_shift_1a', 'F-1-1-4': 'cost_1a',
            'F-1-2-1': 't_detour_1b', 'F-1-2-2': 't_cong_1b', 'F-1-2-3': 't_shift_1b', 'F-1-2-4': 'cost_1b',
            'F-2-1-1': 't_detour_2a', 'F-2-1-2': 't_cong_2a', 'F-2-1-3': 't_shift_2a', 'F-2-1-4': 'cost_2a',
            'F-2-2-1': 't_detour_2b', 'F-2-2-2': 't_cong_2b', 'F-2-2-3': 't_shift_2b', 'F-2-2-4': 'cost_2b',
            'F-3-1-1': 't_detour_3a', 'F-3-1-2': 't_cong_3a', 'F-3-1-3': 't_shift_3a', 'F-3-1-4': 'cost_3a',
            'F-3-2-1': 't_detour_3b', 'F-3-2-2': 't_cong_3b', 'F-3-2-3': 't_shift_3b', 'F-3-2-4': 'cost_3b',
            'F-4-1-1': 't_detour_4a', 'F-4-1-2': 't_cong_4a', 'F-4-1-3': 't_shift_4a', 'F-4-1-4': 'cost_4a',
            'F-4-2-1': 't_detour_4b', 'F-4-2-2': 't_cong_4b', 'F-4-2-3': 't_shift_4b', 'F-4-2-4': 'cost_4b'}

sample = sample.rename(columns=col_name).reset_index(drop=True)

sample['is_driver'] = sample['is_driver'].map({'Yes': 1, 'No': 0})
sample['age'] = sample['age'].map({'Below 18\t': 1, '18-25': 2, '26-35': 3, '36-45': 4, '45-60': 5, 'above 60': 6})
sample['gender'] = sample['gender'].map({'Male': 1, 'Female': 2, 'Non-binary / third gender': 3,
                                         'Prefer not to say': 4})
sample['occ'] = sample['occ'].map({'Student at UC Berkeley': 1, 'Faculty at UC Berkeley': 2, 'Non UC Berkeley': 3})
sample['income'] = sample['income'].map({'Less than $50k': 1, ' $50k-150k': 2, 'Greater than $150k': 3})
sample['hh_size'] = sample['hh_size'].map({'1': 1, '2': 2, '3': 3, '4': 4, '>=5': 5})
sample['have_driven'] = sample['have_driven'].map({'Yes': 1, 'No': 0})
sample['trip_purp'] = sample['trip_purp'].map({'Work': 1, 'School': 2, 'Shopping': 3, 'Leisure': 4, 'Other': 5})
sample['start_time'] = [int(t) if t.isdigit() else '' for t in sample['start_time']]
sample['trip_freq'] = sample['trip_freq'].map({'1': 1, '2': 2, '3': 3, '4': 4, '>=5': 5})
for i in range(1, 5):
    sample[f'choice_{i}'] = sample[f'choice_{i}'].map({'Alternative A': 1, 'Alternative B': 2, 'Alternative C': 3})

In [5]:
# If we only analyze drivers' choice
# sample = sample[sample['is_driver']==1].reset_index(drop=True)

In [6]:
sample.insert(0, 'id', [i+1 for i in sample.index])

In [7]:
sample

,id,is_driver,age,gender,occ,income,hh_size,have_driven,trip_purp,start_time,...,t_shift_3b,cost_3b,t_detour_4a,t_cong_4a,t_shift_4a,cost_4a,t_detour_4b,t_cong_4b,t_shift_4b,cost_4b
0,1,1,3,3,3,2,5,1,1,,...,60.0,10.0,65.0,10.0,45.0,0.0,35.0,60.0,0.0,20.0
1,2,1,4,3,3,2,3,0,1,,...,0.0,0.0,65.0,60.0,0.0,10.0,25.0,10.0,45.0,10.0
2,3,1,2,2,1,1,2,0,5,,...,0.0,0.0,45.0,60.0,45.0,4.0,45.0,60.0,0.0,20.0
3,4,1,1,2,2,1,4,0,2,,...,45.0,10.0,25.0,10.0,15.0,0.0,45.0,90.0,0.0,0.0
4,5,0,5,2,3,1,4,0,5,,...,45.0,15.0,35.0,90.0,15.0,10.0,55.0,25.0,45.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,71,1,2,1,1,1,1,1,4,20,...,30.0,0.0,15.0,25.0,-60.0,4.0,30.0,10.0,60.0,10.0
71,72,1,2,2,1,1,1,0,4,15,...,-60.0,20.0,60.0,10.0,90.0,10.0,15.0,25.0,-30.0,0.0
72,73,1,2,1,1,1,1,0,2,9,...,0.0,4.0,15.0,60.0,90.0,0.0,5.0,0.0,60.0,15.0
73,74,1,3,1,1,1,2,1,4,15,...,-60.0,15.0,15.0,60.0,-90.0,15.0,15.0,60.0,-60.0,0.0


In [8]:
sample_long = sample.iloc[:, :11]
sample_long = sample_long.loc[sample_long.index.repeat(12)].reset_index(drop=True)

t_d, t_c, t_s, c, choice = [[] for i in range(5)]
for i in sample_long.index:
    y = i//12
    if (i%12)%3 == 0:
        x = int((i%12)/3*8)
        t_d.append(sample.iloc[y, 15+x])
        t_c.append(sample.iloc[y, 16+x])
        t_s.append(sample.iloc[y, 17+x])
        c.append(sample.iloc[y, 18+x])
    elif (i%12)%3 == 1:
        x = int((i%12-1)/3*8)
        t_d.append(sample.iloc[y, 19+x])
        t_c.append(sample.iloc[y, 20+x])
        t_s.append(sample.iloc[y, 21+x])
        c.append(sample.iloc[y, 22+x])
    else:
        t_d.append(0)
        t_c.append(0)
        t_s.append(0)
        c.append(0)

sample_long['t_detour'] = t_d
sample_long['t_cong'] = t_c
sample_long['t_shift'] = t_s
sample_long['cost'] = c

for i in sample.index:
    temp = [0 for a in range(12)]
    for j in range(4):
        temp[3*j+sample.iloc[i, 11+j]-1] = 1
    choice.extend(temp)

sample_long['choice'] = choice
sample_long.insert(1, 'alt', [i%3+1 for i in sample_long.index])
sample_long.insert(2, 'sit', [i//3+1 for i in sample_long.index])

In [9]:
sample_long.head(12)

,id,alt,sit,is_driver,age,gender,occ,income,hh_size,have_driven,trip_purp,start_time,trip_freq,t_detour,t_cong,t_shift,cost,choice
0,1,1,1,1,3,3,3,2,5,1,1,,1,35.0,60.0,30.0,20.0,0
1,1,2,1,1,3,3,3,2,5,1,1,,1,45.0,40.0,60.0,10.0,1
2,1,3,1,1,3,3,3,2,5,1,1,,1,0.0,0.0,0.0,0.0,0
3,1,1,2,1,3,3,3,2,5,1,1,,1,25.0,60.0,15.0,15.0,0
4,1,2,2,1,3,3,3,2,5,1,1,,1,45.0,60.0,45.0,0.0,1
5,1,3,2,1,3,3,3,2,5,1,1,,1,0.0,0.0,0.0,0.0,0
6,1,1,3,1,3,3,3,2,5,1,1,,1,65.0,40.0,45.0,20.0,1
7,1,2,3,1,3,3,3,2,5,1,1,,1,35.0,40.0,60.0,10.0,0
8,1,3,3,1,3,3,3,2,5,1,1,,1,0.0,0.0,0.0,0.0,0
9,1,1,4,1,3,3,3,2,5,1,1,,1,65.0,10.0,45.0,0.0,0


# Model formulation & parameter estimation

We noticed that the parameter for $t_\textit{shift}$ is non-significant. Possible reasons include: a) respondents are less sensitive to this variable compared with other variables; 2) respondents generally ignored this variable when answering the survey. We finally used different parameters for positive $t_\textit{shift}$ (depart later) and negative $t_\textit{shift}$ (depart earlier).

In [10]:
sample_long['pos_shift'] = [t if t>0 else 0 for t in t_s]
sample_long['neg_shift'] = [-t if t<0 else 0 for t in t_s]

In [11]:
sample_long['cost_low'] = [i if j==1 else 0 for i, j in zip(c, sample_long['income'])]
sample_long['cost_high'] = [i if (j==2 or j==3) else 0 for i, j in zip(c, sample_long['income'])]

## MNL model

In [12]:
basic_spec = OrderedDict()
basic_names = OrderedDict()

basic_spec['intercept'] = [3]
basic_names['intercept'] = ['ASC (canceling)']

basic_spec['t_detour'] = [[1, 2]]
basic_names['t_detour'] = ['t_detour (min)']

basic_spec['t_cong'] = [[1, 2]]
basic_names['t_cong'] = ['t_congestion (min)']

basic_spec['pos_shift'] = [[1, 2]]
basic_names['pos_shift'] = ['t_shift_late (min)']

basic_spec['neg_shift'] = [[1, 2]]
basic_names['neg_shift'] = ['t_shift_early (min)']

basic_spec['cost_low'] = [[1, 2]]
basic_names['cost_low'] = ['cost for low-income people ($)']

basic_spec['cost_high'] = [[1, 2]]
basic_names['cost_high'] = ['cost for high-income people ($)']

### MNL-1 (candidate model 1, with two cost parameters)

In [13]:
mnl = pl.create_choice_model(data=sample_long,
                             alt_id_col='alt',
                             obs_id_col='sit',
                             choice_col='choice',
                             specification=basic_spec,
                             model_type='MNL',
                             names=basic_names)

In [14]:
mnl.fit_mle(np.zeros(7))
mnl.get_statsmodels_summary()

Log-likelihood at zero: -329.5837
Initial Log-likelihood: -329.5837
Estimation Time for Point Estimation: 0.01 seconds.
Final log-likelihood: -303.9701


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                      choice   No. Observations:                  300
Model:             Multinomial Logit Model   Df Residuals:                      293
Method:                                MLE   Df Model:                            7
Date:                     Tue, 26 Apr 2022   Pseudo R-squ.:                   0.078
Time:                             23:32:48   Pseudo R-bar-squ.:               0.056
AIC:                               621.940   Log-Likelihood:               -303.970
BIC:                               647.867   LL-Null:                      -329.584
===================================================================================================
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
ASC (canceling)                    -1.8552      0.319     -5.815      0.000      -2.481      -1.230
t_detour (min)                     -0.0181      0.005     -3.733      0.000      -0.028      -0.009
t_congestion (min)                 -0.0174      0.004     -4.335      0.000      -0.025      -0.010
t_shift_late (min)                 -0.0032      0.003     -0.963      0.335      -0.010       0.003
t_shift_early (min)                -0.0004      0.004     -0.099      0.922      -0.008       0.007
cost for low-income people ($)     -0.0539      0.015     -3.580      0.000      -0.083      -0.024
cost for high-income people ($)    -0.0473      0.017     -2.720      0.007      -0.081      -0.013
===================================================================================================
"""

### MNL-2 (candidate model 2, with one cost parameter)

In [15]:
basic_spec2 = OrderedDict()
basic_names2 = OrderedDict()

basic_spec2['intercept'] = [3]
basic_names2['intercept'] = ['ASC (canceling)']

basic_spec2['t_detour'] = [[1, 2]]
basic_names2['t_detour'] = ['t_detour (min)']

basic_spec2['t_cong'] = [[1, 2]]
basic_names2['t_cong'] = ['t_congestion (min)']

basic_spec2['pos_shift'] = [[1, 2]]
basic_names2['pos_shift'] = ['t_shift_late (min)']

basic_spec2['neg_shift'] = [[1, 2]]
basic_names2['neg_shift'] = ['t_shift_early (min)']

basic_spec2['cost'] = [[1, 2]]
basic_names2['cost'] = ['cost ($)']

In [16]:
mnl2 = pl.create_choice_model(data=sample_long,
                              alt_id_col='alt',
                              obs_id_col='sit',
                              choice_col='choice',
                              specification=basic_spec2,
                              model_type='MNL',
                              names=basic_names2)

In [17]:
mnl2.fit_mle(np.zeros(6))
mnl2.get_statsmodels_summary()

Log-likelihood at zero: -329.5837
Initial Log-likelihood: -329.5837
Estimation Time for Point Estimation: 0.01 seconds.
Final log-likelihood: -304.0276


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                      choice   No. Observations:                  300
Model:             Multinomial Logit Model   Df Residuals:                      294
Method:                                MLE   Df Model:                            6
Date:                     Tue, 26 Apr 2022   Pseudo R-squ.:                   0.078
Time:                             23:32:48   Pseudo R-bar-squ.:               0.059
AIC:                               620.055   Log-Likelihood:               -304.028
BIC:                               642.278   LL-Null:                      -329.584
=======================================================================================
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
ASC (canceling)        -1.8562      0.319     -5.819      0.000      -2.481      -1.231
t_detour (min)         -0.0181      0.005     -3.738      0.000      -0.028      -0.009
t_congestion (min)     -0.0173      0.004     -4.322      0.000      -0.025      -0.009
t_shift_late (min)     -0.0033      0.003     -0.986      0.324      -0.010       0.003
t_shift_early (min)    -0.0004      0.004     -0.118      0.906      -0.008       0.007
cost ($)               -0.0513      0.013     -3.975      0.000      -0.077      -0.026
=======================================================================================
"""

## Mixed logit model

### With random parameters

In [18]:
mixed_spec = OrderedDict(basic_spec2)
mixed_names = OrderedDict(basic_names2)

mixvar1 = ['t_detour (min)', 't_congestion (min)']

In [19]:
mixed1 = pl.create_choice_model(data=sample_long,
                                alt_id_col='alt',
                                obs_id_col='sit',
                                choice_col='choice',
                                specification=mixed_spec,
                                model_type='Mixed Logit',
                                names=mixed_names,
                                mixing_id_col='id',
                                mixing_vars=mixvar1)

In [20]:
mixed1.fit_mle(init_vals=np.zeros(8), num_draws=600, seed=1)
mixed1.get_statsmodels_summary()

Log-likelihood at zero: -329.5837
Initial Log-likelihood: -329.5837
Estimation Time for Point Estimation: 3.84 seconds.
Final log-likelihood: -295.4841


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Mixed Logit Model Regression Results                     
==============================================================================
Dep. Variable:                 choice   No. Observations:                  300
Model:              Mixed Logit Model   Df Residuals:                      292
Method:                           MLE   Df Model:                            8
Date:                Tue, 26 Apr 2022   Pseudo R-squ.:                   0.103
Time:                        23:32:52   Pseudo R-bar-squ.:               0.079
AIC:                          606.968   Log-Likelihood:               -295.484
BIC:                          636.598   LL-Null:                      -329.584
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
ASC (canceling)             -2.3772      0.335     -7.100      0.000      -3.033      -1.721
t_detour (min)              -0.0259      0.008     -3.316      0.001      -0.041      -0.011
t_congestion (min)          -0.0264      0.006     -4.123      0.000      -0.039      -0.014
t_shift_late (min)          -0.0046      0.004     -1.256      0.209      -0.012       0.003
t_shift_early (min)          0.0013      0.004      0.301      0.763      -0.007       0.010
cost ($)                    -0.0597      0.015     -3.984      0.000      -0.089      -0.030
Sigma t_detour (min)        -0.0254      0.007     -3.409      0.001      -0.040      -0.011
Sigma t_congestion (min)     0.0246      0.009      2.677      0.007       0.007       0.043
============================================================================================
"""

### With nesting (candidate model 3 & final model)

In [21]:
sample_long['driving_nest'] = [1 if (a==1 or a==2) else 0 for a in sample_long['alt']]

In [22]:
mixed_spec = OrderedDict(basic_spec2)
mixed_names = OrderedDict(basic_names2)

mixed_spec['driving_nest'] = [[1, 2]]
mixed_names['driving_nest'] = ['nest of driving']

mixvar2 = ['ASC (canceling)', 'nest of driving']

In [23]:
mixed2 = pl.create_choice_model(data=sample_long,
                                alt_id_col='alt',
                                obs_id_col='sit',
                                choice_col='choice',
                                specification=mixed_spec,
                                model_type='Mixed Logit',
                                names=mixed_names,
                                mixing_id_col='id',
                                mixing_vars=mixvar2)

In [24]:
mixed2.fit_mle(init_vals=np.zeros(9), num_draws=600, constrained_pos=[6], seed=1)
mixed2.get_statsmodels_summary()

Log-likelihood at zero: -329.5837
Initial Log-likelihood: -329.5837
Estimation Time for Point Estimation: 3.63 seconds.
Final log-likelihood: -286.5471


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Mixed Logit Model Regression Results                     
==============================================================================
Dep. Variable:                 choice   No. Observations:                  300
Model:              Mixed Logit Model   Df Residuals:                      291
Method:                           MLE   Df Model:                            9
Date:                Tue, 26 Apr 2022   Pseudo R-squ.:                   0.131
Time:                        23:32:56   Pseudo R-bar-squ.:               0.103
AIC:                          591.094   Log-Likelihood:               -286.547
BIC:                          624.428   LL-Null:                      -329.584
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
ASC (canceling)          -2.6192      0.392     -6.680      0.000      -3.388      -1.851
t_detour (min)           -0.0252      0.006     -4.273      0.000      -0.037      -0.014
t_congestion (min)       -0.0219      0.004     -5.415      0.000      -0.030      -0.014
t_shift_late (min)       -0.0035      0.004     -0.903      0.366      -0.011       0.004
t_shift_early (min)       0.0006      0.004      0.142      0.887      -0.008       0.009
cost ($)                 -0.0571      0.015     -3.829      0.000      -0.086      -0.028
nest of driving                0        nan        nan        nan         nan         nan
Sigma ASC (canceling)     0.5069      4.046      0.125      0.900      -7.423       8.437
Sigma nest of driving     1.6687      1.221      1.367      0.172      -0.724       4.061
=========================================================================================
"""

Note that if we estimate two $\sigma$ parameters for both nests, one of which would be highly non-significant from 0 (thus we directly constrained one parameter to 0).

The following two model specifications are equivalent and the first one is our final model.

In [25]:
# The first model
mixed_spec = OrderedDict(basic_spec2)
mixed_names = OrderedDict(basic_names2)

mixvar2 = ['ASC (canceling)']

In [26]:
mixed = pl.create_choice_model(data=sample_long,
                               alt_id_col='alt',
                               obs_id_col='sit',
                               choice_col='choice',
                               specification=mixed_spec,
                               model_type='Mixed Logit',
                               names=mixed_names,
                               mixing_id_col='id',
                               mixing_vars=mixvar2)

In [27]:
mixed.fit_mle(init_vals=np.zeros(7), num_draws=600, seed=1)
mixed.get_statsmodels_summary()

Log-likelihood at zero: -329.5837
Initial Log-likelihood: -329.5837
Estimation Time for Point Estimation: 3.27 seconds.
Final log-likelihood: -287.4659


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Mixed Logit Model Regression Results                     
==============================================================================
Dep. Variable:                 choice   No. Observations:                  300
Model:              Mixed Logit Model   Df Residuals:                      293
Method:                           MLE   Df Model:                            7
Date:                Tue, 26 Apr 2022   Pseudo R-squ.:                   0.128
Time:                        23:33:00   Pseudo R-bar-squ.:               0.107
AIC:                          588.932   Log-Likelihood:               -287.466
BIC:                          614.858   LL-Null:                      -329.584
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
ASC (canceling)          -2.6193      0.392     -6.688      0.000      -3.387      -1.852
t_detour (min)           -0.0251      0.006     -4.268      0.000      -0.037      -0.014
t_congestion (min)       -0.0219      0.004     -5.420      0.000      -0.030      -0.014
t_shift_late (min)       -0.0034      0.004     -0.896      0.370      -0.011       0.004
t_shift_early (min)       0.0007      0.004      0.155      0.877      -0.008       0.009
cost ($)                 -0.0572      0.015     -3.848      0.000      -0.086      -0.028
Sigma ASC (canceling)    -1.7171      0.335     -5.118      0.000      -2.375      -1.060
=========================================================================================
"""

In [28]:
# The second model
mixed_spec = OrderedDict(basic_spec2)
mixed_names = OrderedDict(basic_names2)

mixed_spec['driving_nest'] = [[1, 2]]
mixed_names['driving_nest'] = ['nest of driving']

mixvar2 = ['nest of driving']

In [29]:
mixed_eqv = pl.create_choice_model(data=sample_long,
                                   alt_id_col='alt',
                                   obs_id_col='sit',
                                   choice_col='choice',
                                   specification=mixed_spec,
                                   model_type='Mixed Logit',
                                   names=mixed_names,
                                   mixing_id_col='id',
                                   mixing_vars=mixvar2)

In [30]:
mixed_eqv.fit_mle(init_vals=np.zeros(8), num_draws=600, constrained_pos=[6], seed=1)
mixed_eqv.get_statsmodels_summary()

Log-likelihood at zero: -329.5837
Initial Log-likelihood: -329.5837
Estimation Time for Point Estimation: 3.49 seconds.
Final log-likelihood: -287.4659


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Mixed Logit Model Regression Results                     
==============================================================================
Dep. Variable:                 choice   No. Observations:                  300
Model:              Mixed Logit Model   Df Residuals:                      292
Method:                           MLE   Df Model:                            8
Date:                Tue, 26 Apr 2022   Pseudo R-squ.:                   0.128
Time:                        23:33:04   Pseudo R-bar-squ.:               0.104
AIC:                          590.932   Log-Likelihood:               -287.466
BIC:                          620.562   LL-Null:                      -329.584
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
ASC (canceling)          -2.6193      0.392     -6.688      0.000      -3.387      -1.852
t_detour (min)           -0.0251      0.006     -4.268      0.000      -0.037      -0.014
t_congestion (min)       -0.0219      0.004     -5.420      0.000      -0.030      -0.014
t_shift_late (min)       -0.0034      0.004     -0.896      0.370      -0.011       0.004
t_shift_early (min)       0.0007      0.004      0.155      0.877      -0.008       0.009
cost ($)                 -0.0572      0.015     -3.848      0.000      -0.086      -0.028
nest of driving                0        nan        nan        nan         nan         nan
Sigma nest of driving     1.7171      0.335      5.118      0.000       1.060       2.375
=========================================================================================
"""

### With alternative-specific variances

This specification does not behave well because the $\sigma$'s for routes 1 and 2 are very non-significant.

In [31]:
mixed_spec = OrderedDict(basic_spec2)
mixed_names = OrderedDict(basic_names2)

mixed_spec['intercept'] = [1, 2, 3]
mixed_names['intercept'] = ['ASC (route 1)', 'ASC (route 2)', 'ASC (canceling)']

mixvar3 = ['ASC (route 1)', 'ASC (route 2)', 'ASC (canceling)']

In [32]:
mixed3 = pl.create_choice_model(data=sample_long,
                                alt_id_col='alt',
                                obs_id_col='sit',
                                choice_col='choice',
                                specification=mixed_spec,
                                model_type='Mixed Logit',
                                names=mixed_names,
                                mixing_id_col='id',
                                mixing_vars=mixvar3)

In [33]:
# The smallest standard deviation, i.e., parameter #9, is constrained to zero
mixed3.fit_mle(init_vals=np.zeros(11), num_draws=600, constrained_pos=[0, 1, 9], seed=1)
mixed3.get_statsmodels_summary()

Log-likelihood at zero: -329.5837
Initial Log-likelihood: -329.5837
Estimation Time for Point Estimation: 3.65 seconds.
Final log-likelihood: -287.1071


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Mixed Logit Model Regression Results                     
==============================================================================
Dep. Variable:                 choice   No. Observations:                  300
Model:              Mixed Logit Model   Df Residuals:                      289
Method:                           MLE   Df Model:                           11
Date:                Tue, 26 Apr 2022   Pseudo R-squ.:                   0.129
Time:                        23:33:08   Pseudo R-bar-squ.:               0.096
AIC:                          596.214   Log-Likelihood:               -287.107
BIC:                          636.956   LL-Null:                      -329.584
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
ASC (route 1)                  0        nan        nan        nan         nan         nan
ASC (route 2)                  0        nan        nan        nan         nan         nan
ASC (canceling)          -2.6136      0.393     -6.653      0.000      -3.384      -1.844
t_detour (min)           -0.0250      0.006     -4.212      0.000      -0.037      -0.013
t_congestion (min)       -0.0219      0.004     -5.375      0.000      -0.030      -0.014
t_shift_late (min)       -0.0034      0.004     -0.887      0.375      -0.011       0.004
t_shift_early (min)       0.0006      0.004      0.145      0.884      -0.008       0.009
cost ($)                 -0.0572      0.015     -3.842      0.000      -0.086      -0.028
Sigma ASC (route 1)      -0.0251      2.468     -0.010      0.992      -4.862       4.812
Sigma ASC (route 2)            0        nan        nan        nan         nan         nan
Sigma ASC (canceling)    -1.7368      0.336     -5.173      0.000      -2.395      -1.079
=========================================================================================
"""

## Nested logit model

In [34]:
nest_membership = OrderedDict()
nest_membership['Nest: canceling'] = [3]
nest_membership['Nest: driving'] = [1, 2]

nested_spec = OrderedDict()
nested_names = OrderedDict()

nested_spec['intercept'] = [3]
nested_names['intercept'] = ['ASC (canceling)']

nested_spec['t_detour'] = [[1, 2]]
nested_names['t_detour'] = ['t_detour (min)']

nested_spec['t_cong'] = [[1, 2]]
nested_names['t_cong'] = ['t_congestion (min)']

nested_spec['pos_shift'] = [[1, 2]]
nested_names['pos_shift'] = ['t_shift_late (min)']

nested_spec['neg_shift'] = [[1, 2]]
nested_names['neg_shift'] = ['t_shift_early (min)']

nested_spec['cost'] = [[1, 2]]
nested_names['cost'] = ['cost ($)']

In [35]:
nested = pl.create_choice_model(data=sample_long,
                                alt_id_col='alt',
                                obs_id_col='sit',
                                choice_col='choice',
                                specification=nested_spec,
                                model_type='Nested Logit',
                                names=nested_names,
                                nest_spec=nest_membership)

In [36]:
init_nests = -1 * np.random.rand(2)
init_coefs = np.zeros(6)
init_vals = np.concatenate((init_nests, init_coefs), axis=0)
nested.fit_mle(init_vals, constrained_pos=[0])
nested.get_statsmodels_summary()

Log-likelihood at zero: -337.5391
Initial Log-likelihood: -343.9431
Estimation Time for Point Estimation: 0.20 seconds.
Final log-likelihood: -299.7564


<class 'statsmodels.iolib.summary.Summary'>
"""
                    Nested Logit Model Regression Results                     
==============================================================================
Dep. Variable:                 choice   No. Observations:                  300
Model:             Nested Logit Model   Df Residuals:                      292
Method:                           MLE   Df Model:                            8
Date:                Tue, 26 Apr 2022   Pseudo R-squ.:                   0.112
Time:                        23:33:08   Pseudo R-bar-squ.:               0.088
AIC:                          615.513   Log-Likelihood:               -299.756
BIC:                          645.143   LL-Null:                      -337.539
=======================================================================================
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Nest: canceling        -0.8107        nan        nan        nan         nan         nan
Nest: driving          -0.9443      1.012     -0.933      0.351      -2.928       1.040
ASC (canceling)        -1.2873      0.334     -3.851      0.000      -1.942      -0.632
t_detour (min)         -0.0074      0.005     -1.369      0.171      -0.018       0.003
t_congestion (min)     -0.0065      0.005     -1.373      0.170      -0.016       0.003
t_shift_late (min)     -0.0011      0.001     -0.732      0.464      -0.004       0.002
t_shift_early (min)    -0.0003      0.002     -0.194      0.846      -0.003       0.003
cost ($)               -0.0180      0.013     -1.364      0.173      -0.044       0.008
=======================================================================================
"""

In [37]:
mu1 = 1 + np.exp(-1*nested.params['Nest: canceling'])
print(f'Estimated Mu for canceling trip = {mu1:.4f}')
mu2 = 1 + np.exp(-1*nested.params['Nest: driving'])
print(f'Estimated Mu for driving = {mu2:.4f}')

Estimated Mu for canceling trip = 3.2496
Estimated Mu for driving = 3.5709
